<a href="https://colab.research.google.com/github/mmtaha/Desafio_Final/blob/main/C%C3%B3pia_de_Codigo_para_transcri%C3%A7%C3%A3o_de_whatsapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -qq ffmpeg
!pip install -q python-docx pydub speechrecognition vosk ffmpeg-python pandas openai-whisper SpeechBrain

In [ ]:
!git clone https://github.com/coqui-ai/STT.git
!pip install -e STT


Cloning into 'STT'...
remote: Enumerating objects: 26333, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 26333 (delta 25), reused 46 (delta 17), pack-reused 26269 (from 1)
Receiving objects: 100% (26333/26333), 54.61 MiB | 24.60 MiB/s, done.
Resolving deltas: 100% (18005/18005), done.
Obtaining file:///content/STT
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of coqui-stt-training to determine which version is compatible with other requirements. This could take a while.
ERROR: Ignored the following versions that require a different python version: 0.52.0 Requires-Py

In [ ]:
!apt-get install -qq ffmpeg
!pip install -q python-docx pydub speechrecognition vosk ffmpeg-python pandas openai-whisper pocketsphinx SpeechBrain google-cloud-speech

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.2/330.2 kB 22.7 MB/s eta 0:00:00


In [ ]:
import os, re, subprocess, json, sys, io, wave
import pandas as pd
from datetime import datetime
from docx import Document
from pydub import AudioSegment
import speech_recognition as sr

# Tenta importar Vosk
try:
    from vosk import Model as VoskModel, KaldiRecognizer
    vosk_available = True
except ImportError:
    vosk_available = False

# Tenta importar OpenAI Whisper
try:
    import whisper
    whisper_available = True
except ImportError:
    whisper_available = False

# Tenta importar SpeechBrain ASR
try:
    from speechbrain.pretrained import EncoderDecoderASR
    speechbrain_available = True
except ImportError:
    speechbrain_available = False

# Tenta importar Google Cloud Speech
try:
    from google.cloud import speech_v1p1beta1 as gcspeech
    google_cloud_available = True
except ImportError:
    google_cloud_available = False

#########################################
# CONFIGURAÇÃO DE CAMINHO
#########################################
# Ajuste os caminhos conforme necessário.
docx_path = "/content/Historico_Conversas_Remilson_Com_Audios.docx"  # Histórico da conversa (DOCX)
audio_folder = "/content"  # Pasta onde os arquivos .opus estão

# Se os áudios estiverem compactados em um ZIP, descompacte-os (opcional):
zip_path = "/content/Conversa do WhatsApp com Remilson Marceneiro.zip"
extracted_audio_folder = "/content/audio_extracted"
if os.path.exists(zip_path):
    import zipfile
    os.makedirs(extracted_audio_folder, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as zf:
        zf.extractall(extracted_audio_folder)
    audio_folder = extracted_audio_folder

# Pasta para salvar arquivos temporários (WAV)
wav_folder = os.path.join(audio_folder, "wav_files")
os.makedirs(wav_folder, exist_ok=True)

#########################################
# FUNÇÃO: Parsing do DOCX (histórico da conversa)
#########################################
def parse_docx_conversation(docx_file):
    try:
        doc = Document(docx_file)
    except Exception as e:
        print(f"Erro ao ler o DOCX: {e}")
        sys.exit(1)
    lines = [para.text for para in doc.paragraphs if para.text.strip()]
    messages = []
    current_date = None
    for line in lines:
        if re.match(r"^\d{2}/\d{2}/\d{4}", line):
            current_date = line.split(" - ")[0].strip()
        elif re.match(r"^\*\*\d{2}:\d{2}\*\*", line):
            parts = line.split(" - ")
            if len(parts) >= 2:
                time = parts[0].replace("**", "").strip()
                if ": " in " - ".join(parts[1:]):
                    sender, message = " - ".join(parts[1:]).split(": ", 1)
                else:
                    sender = "Desconhecido"
                    message = " - ".join(parts[1:])
                timestamp_str = f"{current_date} {time}"
                try:
                    timestamp = datetime.strptime(timestamp_str, "%d/%m/%Y %H:%M")
                except Exception:
                    continue
                messages.append({
                    "timestamp": timestamp,
                    "sender": sender.strip(),
                    "message": message.strip()
                })
    return pd.DataFrame(messages)

print("Processando histórico do DOCX...")
df_messages = parse_docx_conversation(docx_path)
df_messages = df_messages.sort_values(by="timestamp")
print("Total de mensagens processadas:", len(df_messages))

#########################################
# EXTRAÇÃO DOS NOMES DOS ÁUDIOS MENCIONADOS
#########################################
def extract_audio_filename(text):
    m = re.search(r"(PTT-\d{8}-WA\d+\.opus)", text)
    if m:
        return m.group(1)
    return None

df_messages["audio_file"] = df_messages["message"].apply(extract_audio_filename)
audio_files_mentioned = df_messages["audio_file"].dropna().unique()
print("Arquivos de áudio mencionados:", audio_files_mentioned)

available_opus = [f for f in os.listdir(audio_folder) if f.lower().endswith(".opus")]
audio_files_to_process = list(set(audio_files_mentioned) & set(available_opus))
print("Áudios disponíveis para processamento:", audio_files_to_process)

#########################################
# FUNÇÃO: Converter para WAV usando ffmpeg
#########################################
def convert_to_wav(input_file, output_file=None):
    if output_file is None:
        output_file = os.path.splitext(input_file)[0] + ".wav"
    try:
        cmd = ["ffmpeg", "-y", "-i", input_file, output_file]
        subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return output_file
    except Exception as e:
        print(f"Erro na conversão para WAV de {input_file}: {e}")
        return None

#########################################
# DEFINIÇÃO DOS 10 MÉTODOS DE TRANSCRIÇÃO
#########################################

# Método 1: OpenAI Whisper direto no arquivo OPUS (se suportado)
def method_whisper_direct(opus_file):
    try:
        if whisper_available:
            result = whisper.transcribe(opus_file, fp16=False)
            text = result.get("text", "").strip()
            return text if text else None
    except Exception:
        return None

# Método 2: Conversão via ffmpeg para WAV + Whisper
def method_whisper_ffmpeg(opus_file):
    try:
        wav_file = convert_to_wav(opus_file, os.path.join(wav_folder, os.path.basename(opus_file).replace(".opus", "_whisper.wav")))
        if wav_file and whisper_available:
            result = whisper.transcribe(wav_file, fp16=False)
            text = result.get("text", "").strip()
            return text if text else None
    except Exception:
        return None

# Método 3: pydub (formato "opus") + Google Speech Recognition
def method_google_pydub_opus(opus_file):
    try:
        audio = AudioSegment.from_file(opus_file, format="opus")
        buffer = io.BytesIO()
        audio.export(buffer, format="wav")
        buffer.seek(0)
        r = sr.Recognizer()
        with sr.AudioFile(buffer) as source:
            audio_data = r.record(source)
        text = r.recognize_google(audio_data, language="pt-BR").strip()
        return text if text else None
    except Exception:
        return None

# Método 4: Conversão via ffmpeg (mono, 16kHz) + Google Speech Recognition
def method_google_ffmpeg(opus_file):
    try:
        wav_file = convert_to_wav(opus_file, os.path.join(wav_folder, os.path.basename(opus_file).replace(".opus", "_google.wav")))
        if wav_file:
            r = sr.Recognizer()
            with sr.AudioFile(wav_file) as source:
                audio_data = r.record(source)
            text = r.recognize_google(audio_data, language="pt-BR").strip()
            return text if text else None
    except Exception:
        return None

# Método 5: Conversão via ffmpeg + Vosk
def method_vosk_ffmpeg(opus_file):
    try:
        wav_file = convert_to_wav(opus_file, os.path.join(wav_folder, os.path.basename(opus_file).replace(".opus", "_vosk.wav")))
        if not wav_file or not vosk_available:
            return None
        wf = wave.open(wav_file, "rb")
        model_dir = "vosk-model-small-pt-0.3"
        if not os.path.exists(model_dir):
            subprocess.run(["wget", "-q", "https://alphacephei.com/vosk/models/vosk-model-small-pt-0.3.zip"], check=True)
            subprocess.run(["unzip", "-qq", "vosk-model-small-pt-0.3.zip"], check=True)
        model = VoskModel(model_dir)
        rec = KaldiRecognizer(model, wf.getframerate())
        results = []
        while True:
            data = wf.readframes(4000)
            if len(data) == 0:
                break
            if rec.AcceptWaveform(data):
                results.append(json.loads(rec.Result()).get("text", ""))
        results.append(json.loads(rec.FinalResult()).get("text", ""))
        text = " ".join(results).strip()
        return text if text else None
    except Exception:
        return None

# Método 6: pydub (formato "opus") + Whisper (via buffer)
def method_whisper_direct_pydub(opus_file):
    try:
        if whisper_available:
            audio = AudioSegment.from_file(opus_file, format="opus")
            buffer = io.BytesIO()
            audio.export(buffer, format="wav")
            buffer.seek(0)
            temp_wav = os.path.join(wav_folder, os.path.basename(opus_file).replace(".opus", "_temp.wav"))
            with open(temp_wav, "wb") as f:
                f.write(buffer.read())
            result = whisper.transcribe(temp_wav, fp16=False)
            text = result.get("text", "").strip()
            return text if text else None
    except Exception:
        return None

# Método 7: pydub (formato "ogg") + Google Speech Recognition
def method_google_pydub_ogg(opus_file):
    try:
        audio = AudioSegment.from_file(opus_file, format="ogg")
        buffer = io.BytesIO()
        audio.export(buffer, format="wav")
        buffer.seek(0)
        r = sr.Recognizer()
        with sr.AudioFile(buffer) as source:
            audio_data = r.record(source)
        text = r.recognize_google(audio_data, language="pt-BR").strip()
        return text if text else None
    except Exception:
        return None

# Método 8: Conversão via ffmpeg + PocketSphinx
def method_sphinx_ffmpeg(opus_file):
    try:
        wav_file = convert_to_wav(opus_file, os.path.join(wav_folder, os.path.basename(opus_file).replace(".opus", "_sphinx.wav")))
        if wav_file:
            r = sr.Recognizer()
            with sr.AudioFile(wav_file) as source:
                audio_data = r.record(source)
            text = r.recognize_sphinx(audio_data, language="pt-BR").strip()
            return text if text else None
    except Exception:
        return None

# Método 9: pydub (formato "opus") + Vosk (via buffer)
def method_vosk_pydub(opus_file):
    try:
        audio = AudioSegment.from_file(opus_file, format="opus")
        buffer = io.BytesIO()
        audio.export(buffer, format="wav")
        buffer.seek(0)
        wf = wave.open(buffer, "rb")
        model_dir = "vosk-model-small-pt-0.3"
        if not os.path.exists(model_dir):
            subprocess.run(["wget", "-q", "https://alphacephei.com/vosk/models/vosk-model-small-pt-0.3.zip"], check=True)
            subprocess.run(["unzip", "-qq", "vosk-model-small-pt-0.3.zip"], check=True)
        model = VoskModel(model_dir)
        rec = KaldiRecognizer(model, wf.getframerate())
        results = []
        while True:
            data = wf.readframes(4000)
            if len(data) == 0:
                break
            if rec.AcceptWaveform(data):
                results.append(json.loads(rec.Result()).get("text", ""))
        results.append(json.loads(rec.FinalResult()).get("text", ""))
        text = " ".join(results).strip()
        return text if text else None
    except Exception:
        return None

# Método 10: Conversão via ffmpeg + Google Cloud Speech (se configurado)
def method_google_cloud(opus_file):
    try:
        if not google_cloud_available:
            return None
        wav_file = convert_to_wav(opus_file, os.path.join(wav_folder, os.path.basename(opus_file).replace(".opus", "_gc.wav")))
        if wav_file:
            client = gcspeech.SpeechClient()
            with io.open(wav_file, "rb") as f:
                content = f.read()
            audio = gcspeech.RecognitionAudio(content=content)
            config = gcspeech.RecognitionConfig(
                encoding=gcspeech.RecognitionConfig.AudioEncoding.LINEAR16,
                sample_rate_hertz=16000,
                language_code="pt-BR",
            )
            response = client.recognize(config=config, audio=audio)
            texts = [result.alternatives[0].transcript for result in response.results]
            return " ".join(texts).strip() if texts else None
    except Exception:
        return None

# Lista de métodos (nome e função) para transcrição
transcription_methods = [
    ("Whisper Direct", method_whisper_direct),
    ("Whisper via ffmpeg", method_whisper_ffmpeg),
    ("Google via pydub (opus)", method_google_pydub_opus),
    ("Google via ffmpeg", method_google_ffmpeg),
    ("Vosk via ffmpeg", method_vosk_ffmpeg),
    ("Whisper via pydub", method_whisper_direct_pydub),
    ("Google via pydub (ogg)", method_google_pydub_ogg),
    ("PocketSphinx via ffmpeg", method_sphinx_ffmpeg),
    ("Vosk via pydub", method_vosk_pydub),
    ("Google Cloud", method_google_cloud)
]

def robust_transcribe(opus_file):
    for idx, (name, method) in enumerate(transcription_methods, start=1):
        print(f"Tentativa {idx}: usando método '{name}' para {os.path.basename(opus_file)}")
        try:
            result = method(opus_file)
            if result is not None and result.strip() != "":
                print(f"Método '{name}' obteve transcrição: {result}\n")
                return result, name
        except Exception as e:
            print(f"Erro no método '{name}': {e}")
    return "[Falha na transcrição]", None

#########################################
# PROCESSAMENTO DOS ÁUDIOS
#########################################
audio_transcriptions = {}
success_count = 0
for af in audio_files_to_process:
    opus_full_path = os.path.join(audio_folder, af)
    print(f"\nProcessando áudio: {af}")
    transcription, method_used = robust_transcribe(opus_full_path)
    audio_transcriptions[af] = transcription
    if transcription not in ("[Falha na transcrição]", "", None):
        success_count += 1
    print(f"Transcrição final para {af} ({method_used}): {transcription}")

total_audios = len(audio_files_to_process)
success_percent = (success_count / total_audios) * 100 if total_audios > 0 else 0
print(f"\nTaxa de sucesso na transcrição: {success_percent:.1f}% ({success_count}/{total_audios})")

#########################################
# INTEGRAÇÃO COM O HISTÓRICO DA CONVERSA
#########################################
def append_transcription(message):
    m = re.search(r"(PTT-\d{8}-WA\d+\.opus)", message)
    if m:
        file_name = m.group(1)
        transcr = audio_transcriptions.get(file_name, "[Erro na transcrição]")
        return f"{message} - Transcrição: {transcr}"
    return message

df_messages["message"] = df_messages["message"].apply(append_transcription)

#########################################
# RESUMO E CONFIRMAÇÃO PARA GERAR ARQUIVO FINAL
#########################################
print("\nResumo das primeiras 20 mensagens processadas:")
print(df_messages.head(20).to_string(index=False))
print("...\nTotal de mensagens:", len(df_messages))

confirm = input("Deseja gerar o arquivo final CSV com toda a conversa? (s/n): ").strip().lower()
if confirm == "s":
    output_csv = os.path.join(audio_folder, "conversa_transcrita.csv")
    df_messages.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"Arquivo gerado com sucesso: {output_csv}")
else:
    print("Arquivo não gerado.")


Processando histórico do DOCX...
Total de mensagens processadas: 569
Arquivos de áudio mencionados: ['PTT-20231227-WA0034.opus' 'PTT-20231227-WA0048.opus'
 'PTT-20231227-WA0049.opus' 'PTT-20231227-WA0051.opus'
 'PTT-20231227-WA0050.opus' 'PTT-20231227-WA0053.opus'
 'PTT-20231227-WA0052.opus' 'PTT-20231228-WA0004.opus'
 'PTT-20231228-WA0005.opus' 'PTT-20231228-WA0008.opus'
 'PTT-20231228-WA0007.opus' 'PTT-20231228-WA0006.opus'
 'PTT-20231228-WA0010.opus' 'PTT-20231228-WA0009.opus'
 'PTT-20231228-WA0011.opus' 'PTT-20231228-WA0012.opus'
 'PTT-20231228-WA0013.opus' 'PTT-20240110-WA0016.opus'
 'PTT-20240110-WA0018.opus' 'PTT-20240110-WA0017.opus'
 'PTT-20240110-WA0019.opus' 'PTT-20240110-WA0022.opus'
 'PTT-20240110-WA0023.opus' 'PTT-20240110-WA0021.opus'
 'PTT-20240110-WA0024.opus' 'PTT-20240110-WA0027.opus'
 'PTT-20240110-WA0026.opus' 'PTT-20240110-WA0029.opus'
 'PTT-20240110-WA0028.opus' 'PTT-20240113-WA0012.opus'
 'PTT-20240113-WA0013.opus' 'PTT-20240113-WA0014.opus'
 'PTT-20240113-WA001